In [11]:
import numpy as np
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import cobra

In [12]:
model_glucose = cobra.io.read_sbml_model('data/iML1515_het.xml')
model_acetate = cobra.io.read_sbml_model('data/iML1515_het_acetate.xml')

In [13]:
fva_glucose = flux_variability_analysis(model_glucose, model_glucose.reactions)
fva_acetate = flux_variability_analysis(model_acetate, model_acetate.reactions)

# Extract the flux values from the matrices
flux_values_glucose = fva_glucose["maximum"].values
flux_values_acetate = fva_acetate["maximum"].values

# Calculate the matrix of differences
flux_differences = flux_values_acetate - flux_values_glucose

# Create a DataFrame for better visualization
result_matrix = pd.DataFrame(data=flux_differences, index=model_glucose.reactions, columns=["Flux Difference"])

# Add a column for absolute flux differences
result_matrix["Absolute Flux Difference"] = np.abs(result_matrix["Flux Difference"])

# Sort the values by the absolute flux differences in descending order
result_matrix = result_matrix.sort_values(by="Absolute Flux Difference", ascending=False)

# Keep only the top 10 reactions
top_10_reactions = result_matrix.head(10)

# Drop the temporary columns used for sorting
top_10_reactions = top_10_reactions.drop(columns=["Absolute Flux Difference"])

# Save the top 10 reactions to a CSV file
top_10_reactions.to_csv("top_10_flux_differences.csv")

print(top_10_reactions)


                                                    Flux Difference
ATPS4rpp: adp_c + 4.0 h_p + pi_c <=> atp_c + h2...       -37.924311
EX_h2o_e: h2o_e <=>                                      -34.265553
H2Otex: h2o_e <=> h2o_p                                   30.237833
H2Otpp: h2o_p <=> h2o_c                                   30.237833
FRD2: fum_c + mql8_c --> mqn8_c + succ_c                 -23.272483
NADH16pp: 4.0 h_c + nadh_c + q8_c --> 3.0 h_p +...       -23.050357
NADH18pp: 2dmmq8_c + 4.0 h_c + nadh_c --> 2dmmq...       -23.050357
FRD3: 2dmmql8_c + fum_c --> 2dmmq8_c + succ_c            -23.050357
NADH17pp: 4.0 h_c + mqn8_c + nadh_c --> 3.0 h_p...       -23.050357
SUCDi: q8_c + succ_c --> fum_c + q8h2_c                  -22.404552


### Within the list on top of reactions affected by the usage of acetate on the media, we have 5 types of reactions:

1) ATP synthase:

Function: Catalyzes the synthesis of ATP from ADP.

Possible Reduction: Due to changes in the availability of NADH or FADH2 coming from the metabolism of acetate glucose.

2) H2O exchange and H2O transport via diffusion:

Function: Exchange and diffusion of water.

Possible Reduction: Changes in the carbon source may affect the ion transport and energy metabolism.

3) Fumarate reductase:

Function: Catalyzes the conversion of fumarate to succinate in the electron transport chain.

Possible Reduction: The reduction may be associated with the availability of electron donors and acceptors in the ETC when using acetate.

4) NADH dehydrogenase:

Function: These enzymes are involved in the transfer of electrons from NADH to the ETC, generating the proton gradient.

Possible Reduction: Flux reduction may be due to the availability of NADH and/or specific electron carriers.

5) Succinate dehydrogenase:

Function: Catalyzes the conversion of succinate to fumarate in the TCA cycle. It also participates in the ETC.

Possible Reduction: A lower flux might be due to alterations in the TCA cycle activity, mainly because of the usage of a different carbon source (less efficient in this case).